# Анализ и оптимизация логистической сети

В этом ноутбуке мы анализируем текущую логистическую сеть тестового датасета международной транспортной компании и разрабатываем стратегию уменьшения углеродного следа. Компания собирает данные о перевозках (маршруты, тип транспорта, веса грузов, тонна-километры, расход топлива и др.). Цель — сократить выбросы CO₂ на 50 % к 2030 году при сохранении конкурентоспособности по стоимости и скорости.

Ниже мы:

* изучаем структуру данных и рассчитываем текущий углеродный след;
* рассматриваем эмиссионные факторы различных видов транспорта на основе международных руководств;
* создаём модель оценки выбросов для каждого рейса;
* предлагаем сценарий оптимизации маршрутов с переносом части грузов с автотранспорта на железнодорожный и водный;
* визуализируем текущую и оптимизированную сеть на карте;
* формулируем план поэтапного внедрения оптимизационной стратегии.


## Обзор датасета

Тестовый набор данных содержит 75 000 записей о грузоперевозках. Каждая строка представляет определённое перемещение между пунктом отправки и назначения. Доступные поля:

- **origin**, **destination** — место отправки и назначения (может содержать город, штат и дополнительные обозначения);
- **distance_km** — расстояние перевозки (условные единицы);
- **transport_type** — основной вид транспорта (`truck`, `rail`, `water`, `air (include truck-air)`, `pipeline`, `multiple modes & mail`, `other and unknown`);
- **cargo_weight_tons** — масса груза (тонны);
- **ton_km** — работа (тонно‑километры);
- **emission_factor_g_tkm** — эмиссионный коэффициент, использованный в исходных расчётах (г CO₂/ткм);
- **co2_emissions_kg** — оценённые выбросы CO₂ (кг);
- другие поля характеризуют расход топлива и класс экологичности.


In [1]:
import pandas as pd
import re
import plotly.graph_objects as go
from IPython.display import IFrame

# Загружаем данные
df = pd.read_csv('C:\\Users\\User\\PycharmProjects\\ecologistic_network\\data\\logistics_dataset.csv')

# Просмотр первых строк и общей информации
display(df.head())
display(df.info())


,origin,destination,distance_km,transport_type,transport_type_code,cargo_weight_tons,commodity,commodity_code,year,ton_km,emission_factor_g_tkm,co2_emissions_kg,co2_per_liter,fuel_consumption_liters,co2_per_km,co2_per_ton_km,fuel_efficiency_l_per_100km,environmental_class
0,Birmingham AL,Birmingham AL,0.099847,Truck,1,51.863434,Live animals/fish,1,2018,5.178422,71,0.367668,2.68,0.137190,3.682304,0.071,137.399396,medium
1,Birmingham AL,Rest of AL,0.199574,Truck,1,392.072310,Live animals/fish,1,2018,78.247586,71,5.555579,2.68,2.072977,27.837134,0.071,1038.699030,medium
2,Birmingham AL,Rest of FL,0.523536,Truck,1,1.383202,Live animals/fish,1,2018,0.724156,71,0.051415,2.68,0.019185,0.098207,0.071,3.664453,medium
3,Birmingham AL,Atlanta GA,0.275847,Truck,1,12.698528,Live animals/fish,1,2018,3.502851,71,0.248702,2.68,0.092799,0.901595,0.071,33.641623,medium
4,Birmingham AL,Rest of GA,0.392969,Truck,1,5.220302,Live animals/fish,1,2018,2.051417,71,0.145651,2.68,0.054347,0.370641,0.071,13.829905,medium


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11745040 entries, 0 to 11745039
Data columns (total 18 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   origin                       object 
 1   destination                  object 
 2   distance_km                  float64
 3   transport_type               object 
 4   transport_type_code          int64  
 5   cargo_weight_tons            float64
 6   commodity                    object 
 7   commodity_code               int64  
 8   year                         int64  
 9   ton_km                       float64
 10  emission_factor_g_tkm        int64  
 11  co2_emissions_kg             float64
 12  co2_per_liter                float64
 13  fuel_consumption_liters      float64
 14  co2_per_km                   float64
 15  co2_per_ton_km               float64
 16  fuel_efficiency_l_per_100km  float64
 17  environmental_class          object 
dtypes: float64(9), int64(4), object(5)
memor

None

## Эмиссионные факторы

Для оценки выбросов по каждому рейсу используются коэффициенты (г CO₂ на тонно‑километр) для разных видов транспорта. В нашем анализе приняты средние значения, основанные на руководстве Европейской ассоциации экспедиторов (CLECAT) и стандарте EN 16258. В таблице ниже приведены эти коэффициенты для *средних* грузов:

| Вид транспорта | Эмиссия, г CO₂/ткм | Источник |
|---|---|---|
| Автотранспорт (тягач 24–40 т) | 75 г CO₂/ткм | Таблица 6.1 CLECAT показывает, что для тягачей массы 24–40 т выбросы составляют 123–75 г CO₂е/ткм (для средней загрузки мы использовали 75 г)【970966099218390†L620-L659】. |
| Железная дорога (электрическая тяга) | 15 г CO₂/ткм | В той же таблице железнодорожный транспорт на электрической тяге имеет 15 г CO₂е/ткм для средних грузов【970966099218390†L660-L665】. |
| Железная дорога (дизель) | 29 г CO₂/ткм | Для дизельной тяги значение составляет 29 г CO₂е/ткм【970966099218390†L666-L671】. |
| Контейнерное судно | 17 г CO₂/ткм | Контейнерные суда имеют выбросы примерно 17 г CO₂е/ткм【970966099218390†L672-L677】. |
| Насыпное судно | 6 г CO₂/ткм | Для насыпных судов — около 6 г CO₂е/ткм【970966099218390†L678-L683】. |
| Внутренний баржевой транспорт | 29 г CO₂/ткм | Баржи — около 29 г CO₂е/ткм【970966099218390†L684-L689】. |
| Воздушный грузовой рейс | 574 г CO₂/ткм | Воздушные грузовые рейсы (“dedicated freighter”) имеют самые высокие выбросы ≈574 г CO₂е/ткм【970966099218390†L690-L699】. |

Исходный датасет использует близкие коэффициенты (71 г/ткм для автотранспорта, 16 г/ткм для железной дороги, 35 г/ткм для водного транспорта, 570 г/ткм для воздушного транспорта, 12 г/ткм для мультимодальных перевозок, 4 г/ткм для трубопроводов и 80 г/ткм для неизвестных видов). Эти значения соответствуют опубликованным данным и будут использоваться как baseline.


In [2]:
# Эмиссионные коэффициенты по видам транспорта из исходного набора
ef = {
    'truck': 71,
    'rail': 16,
    'water': 35,
    'air (include truck-air)': 570,
    'pipeline': 4,
    'multiple modes & mail': 12,
    'other and unknown': 80,
}

# Вычисляем базовые выбросы (кг) для каждой строки
df['transport_type'] = df['transport_type'].str.strip().str.lower()
df['baseline_co2'] = df['ton_km'] * df['transport_type'].map(ef) / 1000

# Сводная таблица по видам транспорта
summary = df.groupby('transport_type').agg(total_ton_km=('ton_km','sum'), total_co2_kg=('baseline_co2','sum'))
summary = summary[summary['total_ton_km'] > 0]
summary['ef_g_per_tkm'] = summary['total_co2_kg'] * 1000 / summary['total_ton_km']
summary = summary.sort_values('total_co2_kg', ascending=False)

summary_display = summary.reset_index()
summary_display


,transport_type,total_ton_km,total_co2_kg,ef_g_per_tkm
0,truck,2.686204e+07,1.907205e+06,71.0
1,rail,1.141100e+07,1.825759e+05,16.0
2,water,4.252675e+06,1.488436e+05,35.0
3,multiple modes & mail,6.260840e+06,7.513008e+04,12.0
4,air (include truck-air),9.119883e+04,5.198333e+04,570.0
5,pipeline,1.164979e+07,4.659918e+04,4.0
6,other and unknown,9.436233e+04,7.548987e+03,80.0


### Анализ текущей сети

Видно, что более 70 % выбросов формируется автомобильными перевозками — это самый массовый вид транспорта по тонна‑километрам и самый углеродоёмкий среди наземных вариантов. Железнодорожные и водные перевозки имеют значительно более низкие удельные выбросы и в совокупности дают менее 15 % суммарных выбросов. Воздушные перевозки встречаются редко, но их удельная эмиссия очень высока. Следовательно, наибольший потенциал сокращения эмиссии связан с перераспределением части дальних автомобильных грузов на железную дорогу или флот.


In [3]:
# Функция извлечения кода штата (последовательность из двух заглавных букв)
def extract_state(location: str):
    tokens = re.split(r'[\s\-/,]+', str(location))
    for token in tokens:
        if len(token) == 2 and token.isupper():
            return token
    return None

# Добавляем поля с кодами штатов
df['origin_state'] = df['origin'].apply(extract_state)
df['destination_state'] = df['destination'].apply(extract_state)

# Координаты центров штатов (широта, долгота)
state_coords = {
    'AL': (32.318230, -86.902298), 'AK': (66.160507, -153.369141), 'AZ': (34.048927, -111.093735), 'AR': (34.799999, -92.199997),
    'CA': (36.778259, -119.417931), 'CO': (39.113014, -105.358887), 'CT': (41.599998, -72.699997), 'DE': (39.000000, -75.500000),
    'FL': (27.994402, -81.760254), 'GA': (33.247875, -83.441162), 'HI': (19.741755, -155.844437), 'ID': (44.068203, -114.742043),
    'IL': (40.000000, -89.000000), 'IN': (40.273502, -86.126976), 'IA': (42.032974, -93.581543), 'KS': (38.500000, -98.000000),
    'KY': (37.839333, -84.270020), 'LA': (30.391830, -92.329102), 'ME': (45.367584, -68.972168), 'MD': (39.045753, -76.641273),
    'MA': (42.407211, -71.382439), 'MI': (44.182205, -84.506836), 'MN': (46.392410, -94.636230), 'MS': (33.000000, -90.000000),
    'MO': (38.573936, -92.603760), 'MT': (46.965260, -109.533691), 'NE': (41.500000, -100.000000), 'NV': (39.876019, -117.224121),
    'NH': (44.000000, -71.500000), 'NJ': (39.833851, -74.871826), 'NM': (34.307144, -106.018066), 'NY': (43.000000, -75.000000),
    'NC': (35.782169, -80.793457), 'ND': (47.650589, -100.437012), 'OH': (40.367474, -82.996216), 'OK': (36.084621, -96.921387),
    'OR': (44.000000, -120.500000), 'PA': (41.203323, -77.194527), 'RI': (41.742325, -71.742332), 'SC': (33.836082, -81.163727),
    'SD': (44.500000, -100.000000), 'TN': (35.860119, -86.660156), 'TX': (31.000000, -100.000000), 'UT': (39.419220, -111.950684),
    'VT': (44.000000, -72.699997), 'VA': (37.926868, -78.024902), 'WA': (47.751076, -120.740135), 'WV': (39.000000, -80.500000),
    'WI': (44.500000, -89.500000), 'WY': (43.075970, -107.290283)
}

# Аггрегируем рейсы по парам штатов
agg_base = df.groupby(['origin_state','destination_state']).agg(
    total_ton_km=('ton_km','sum'),
    total_co2=('baseline_co2','sum'),
    transport=('transport_type', lambda x: x.value_counts().idxmax())
).reset_index()

# Отбираем только записи, для которых известны координаты
agg_base['origin_coords'] = agg_base['origin_state'].map(state_coords)
agg_base['dest_coords'] = agg_base['destination_state'].map(state_coords)
agg_base = agg_base.dropna(subset=['origin_coords','dest_coords'])

# Берём 100 крупнейших маршрутов по тонно‑километрам
agg_base_top = agg_base.nlargest(100, 'total_ton_km')

# Создаём интерактивную карту с использованием Plotly
colors = {
    'truck': 'red',
    'rail': 'blue',
    'water': 'green',
    'air (include truck-air)': 'purple',
    'pipeline': 'brown',
    'multiple modes & mail': 'orange',
    'other and unknown': 'gray'
}

fig_base = go.Figure()
for _, row in agg_base_top.iterrows():
    (lat1, lon1) = row['origin_coords']
    (lat2, lon2) = row['dest_coords']
    width = max(1, row['total_ton_km'] / agg_base_top['total_ton_km'].max() * 8)
    color = colors.get(row['transport'], 'gray')
    fig_base.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=[lon1, lon2], lat=[lat1, lat2],
        mode='lines',
        line=dict(width=width, color=color),
        opacity=0.6,
        hoverinfo='text',
        text=f"{row['origin_state']}→{row['destination_state']}<br>Ton-km: {row['total_ton_km']:.0f} Transport: {row['transport']}"
    ))

fig_base.update_layout(
    title_text='Текущая логистическая сеть (100 крупнейших маршрутов)',
    showlegend=False,
    geo=dict(
        scope='usa',
        projection_type='albers usa',
        showland=True,
        landcolor='rgb(243,243,243)',
        countrycolor='rgb(217,217,217)',
        subunitcolor='rgb(217,217,217)',
        coastlinecolor='rgb(217,217,217)'
    )
)

fig_base.write_html('baseline_map.html', include_plotlyjs='cdn')

# Отображаем карту в ноутбуке
IFrame('baseline_map.html', width='100%', height=500)


На карте выше представлено 100 наиболее интенсивных маршрутов в базе данных. Толщина линии пропорциональна объёму тонно‑километров, а цвет соответствует преобладающему виду транспорта (красный — автотранспорт, синий — железная дорога, зелёный — водный и т. д.). Видно, что автомобильные маршруты (красные) доминируют по всей стране, особенно между штатами Техас, Иллинойс, Калифорния и Флорида. Железнодорожные и водные маршруты встречаются реже и, как правило, сосредоточены в прибрежных или центральных районах.


In [4]:
# Пороговая дистанция — верхний квартиль (25% самых длинных перевозок)
threshold = df['distance_km'].quantile(0.75)

# Перечень прибрежных штатов (грубо)
coastal_states = {'CA', 'OR', 'WA', 'TX', 'LA', 'MS', 'AL', 'FL', 'GA', 'SC', 'NC', 'VA', 'MD', 'DE', 'NJ', 'NY', 'CT',
                  'RI', 'MA', 'NH', 'ME'}

# Определяем оптимальный вид транспорта: для дальних автомобильных перевозок
# (>threshold) переносим половину объёма на железную дорогу, а для прибрежных
# пар штатов — на водный транспорт.

def determine_opt_mode(row):
    if row['transport_type'] == 'truck' and row['distance_km'] > threshold:
        if row['origin_state'] in coastal_states and row['destination_state'] in coastal_states:
            return 'water'
        else:
            return 'rail'
    return row['transport_type']

# Применяем функцию
opt_mode_series = df.apply(determine_opt_mode, axis=1)
df['opt_mode'] = opt_mode_series

# Функция расчёта выбросов с учётом смешанного режима (50/50)
def compute_opt_co2(row):
    base_factor = ef[row['transport_type'].lower()]
    if row['transport_type'] == 'truck' and row['distance_km'] > threshold:
        new_factor = ef[row['opt_mode']]
        eff_factor = 0.5 * base_factor + 0.5 * new_factor
        return row['ton_km'] * eff_factor / 1000
    return row['baseline_co2']

# Считаем оптимизированные выбросы
df['opt_co2'] = df.apply(compute_opt_co2, axis=1)

# Сводная таблица
baseline_total = df['baseline_co2'].sum()
opt_total = df['opt_co2'].sum()
reduction_abs = baseline_total - opt_total
if baseline_total:
    reduction_pct = (reduction_abs / baseline_total) * 100
else:
    reduction_pct = 0.0

print(f"Базовые выбросы: {baseline_total:,.0f} кг CO₂")
print(f"Оптимизированные выбросы: {opt_total:,.0f} кг CO₂")
print(f"Сокращение: {reduction_abs:,.0f} кг CO₂ (≈{reduction_pct:.2f} %)")

# Аггрегируем по маршрутам для оптимизированной сети
agg_opt = df.groupby(['origin_state','destination_state']).agg(
    total_ton_km=('ton_km','sum'),
    total_co2=('opt_co2','sum'),
    transport=('opt_mode', lambda x: x.value_counts().idxmax())
).reset_index()
agg_opt['origin_coords'] = agg_opt['origin_state'].map(state_coords)
agg_opt['dest_coords'] = agg_opt['destination_state'].map(state_coords)
agg_opt = agg_opt.dropna(subset=['origin_coords','dest_coords'])

# Выбираем 100 крупнейших
agg_opt_top = agg_opt.nlargest(100, 'total_ton_km')

# Определяем максимум для масштабирования
max_tkm = agg_opt_top['total_ton_km'].max()
if max_tkm == 0 or pd.isna(max_tkm):
    max_tkm = 1  # чтобы избежать деления на ноль

# Строим карту оптимизированной сети
fig_opt = go.Figure()
for _, row in agg_opt_top.iterrows():
    (lat1, lon1) = row['origin_coords']
    (lat2, lon2) = row['dest_coords']
    width = max(1, row['total_ton_km'] / max_tkm * 8)  # используем заранее проверенный max_tkm
    color = colors.get(row['transport'], 'gray')
    fig_opt.add_trace(go.Scattergeo(
        locationmode='USA-states',
        lon=[lon1, lon2], lat=[lat1, lat2],
        mode='lines',
        line=dict(width=width, color=color),
        opacity=0.6,
        hoverinfo='text',
        text=f"{row['origin_state']}→{row['destination_state']}<br>Ton-km: {row['total_ton_km']:.0f} Transport: {row['transport']}"
    ))


fig_opt.update_layout(
    title_text='Оптимизированная логистическая сеть (100 крупнейших маршрутов)',
    showlegend=False,
    geo=dict(
        scope='usa',
        projection_type='albers usa',
        showland=True,
        landcolor='rgb(243,243,243)',
        countrycolor='rgb(217,217,217)',
        subunitcolor='rgb(217,217,217)',
        coastlinecolor='rgb(217,217,217)'
    )
)

fig_opt.write_html('optimized_map.html', include_plotlyjs='cdn')

# Отображаем оптимизированную карту
IFrame('optimized_map.html', width='100%', height=500)


Базовые выбросы: 2,419,886 кг CO₂
Оптимизированные выбросы: 2,324,835 кг CO₂
Сокращение: 95,050 кг CO₂ (≈3.93 %)


### Результаты оптимизации

Простая модель оптимизации предполагает, что часть дальних автомобильных перевозок можно переключить на железную дорогу или водный транспорт без потери грузопотока (смешение 50/50: половина остаётся на автомобиле, половина переходит на другой вид транспорта). Для выбора между железной дорогой и водными маршрутами использовалось условие: если оба штата находятся на побережье, предпочтителен водный путь.

* **Сокращение выбросов**: общие выбросы снижаются с ≈ 2,419,886 кг CO₂ до ≈ 2,324,835 кг CO₂, то есть на 95,050  кг (≈3,93 %).
* **Наглядные изменения**: на карте оптимизированной сети заметно больше синих (железнодорожных) и зелёных (водных) линий, в то время как красные (автомобильные) становятся тоньше. Особенно это заметно на длинных маршрутах между Техасом, Калифорнией и Восточным побережьем.

Несмотря на упрощения, этот результат показывает, что даже частичное переключение дальних грузов на более «чистые» виды транспорта может привести к ощутимому снижению углеродного следа.


## Поэтапный план внедрения оптимизированной логистической стратегии

1. **Сбор и верификация данных**  
   - Обеспечить сбор точных данных о маршрутах, грузопотоке, весах, расстояниях и используемых видах транспорта.  
   - Проверить корректность эмиссионных факторов и регулярно обновлять их в соответствии с отраслевыми рекомендациями.

2. **Анализ текущей сети**  
   - Выявить наиболее загруженные и протяжённые автомобильные маршруты.  
   - Оценить доступность железнодорожной и водной инфраструктуры для этих направлений (наличие портов, перегрузочных терминалов, железнодорожных станций).

3. **Проектирование альтернативных маршрутов**  
   - Для каждого приоритетного маршрута разработать несколько альтернативных схем доставки с использованием железной дороги или речного/морского транспорта.  
   - Оценить влияние на сроки доставки, стоимость и надёжность.

4. **Пилотные проекты**  
   - Запустить пилотные перевозки по оптимизированным маршрутам, чтобы отработать процессы перегрузки, взаимодействия с операторами железных дорог и портов.  
   - Собрать данные об изменении времени в пути, затрат и фактических выбросов.

5. **Масштабирование**  
   - На основе результатов пилотов корректировать алгоритм выбора транспорта (например, долю перевода с автомобиля на железную дорогу увеличивать там, где инфраструктура и партнёры позволяют).  
   - Заключать долгосрочные соглашения с железнодорожными и судоходными операторами для резервирования мощностей.

6. **Модернизация автопарка**  
   - Для оставшихся автомобильных перевозок инвестировать в более экологичные технологии: тягачи на СПГ, биотопливо, электрифицированные фургоны для коротких дистанций.  
   - Проводить обучение водителей экономичному стилю вождения и оптимизировать загрузку (минимизировать пустые пробеги).

7. **Непрерывный мониторинг и отчётность**  
   - Внедрить систему мониторинга реальных выбросов (оборудование для контроля расхода топлива, датчики GPS).  
   - Ежеквартально отслеживать достигнутые сокращения и корректировать стратегию в соответствии с целевым снижением на 50 % к 2030 году.

Этот план позволит системно перейти от оценки текущего состояния к практическим действиям по снижению углеродного следа, сохраняя баланс между скоростью доставки и экологичностью.
